In [1]:
import os
import cv2
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
# from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class ImitationDataset(Dataset):
    def __init__(self, rgb_dir, seg_dir, log_path, transforms=None):
        self.rgb_dir = rgb_dir
        self.seg_dir = seg_dir
        self.log_path = log_path
        self.transform = transforms
        
        with open(log_path, 'r') as f:
            self.log_data = json.load(f)
            
    def __len__(self):
        return len(self.log_data)
    
    def __getitem__(self, index):
        rgb_image_path = os.path.join(self.rgb_dir, f"{index:05d}.png")
        seg_image_path = os.path.join(self.seg_dir, f"{index:05d}.png")
        
        rgb_image = Image.open(rgb_image_path)
        
        if self.transform:
            rgb_tensor = self.transform(rgb_image)
        else:
            rgb_tensor = transforms.ToTensor()(rgb_image)
            
        seg_image = np.array(Image.open(seg_image_path))
        
        lane_mask = np.all(seg_image == [0, 255, 0], axis=2).astype(np.uint8)
        obs_mask = np.all(seg_image == [255, 0, 0], axis=2).astype(np.uint8)
        
        seg_tensor = torch.tensor(np.stack([lane_mask, obs_mask], axis=0), dtype=torch.float32)
        
        if seg_tensor.shape[1:] != rgb_tensor.shape[1:]:
            seg_tensor = F.interpolate(
                seg_tensor.unsqueeze(0),
                size=rgb_tensor.shape[1:],
                mode='nearest'
            ).squeeze(0)
        
        input_tensor = torch.cat([rgb_tensor, seg_tensor], dim=0)
                        
        control = self.log_data[index]
        control_tensor = torch.tensor([
            control['steer'],
            control['throttle'],
            control['brake']
        ], dtype=torch.float32)
        
        return input_tensor, control_tensor
        


In [ ]:
# class ImitationCNN(nn.Module):
#     def __init__(self):
#         super(ImitationCNN, self).__init__()
        
#         self.conv1 = nn.Conv2d(5, 32, kernel_size=5, stride=2, padding=2)  
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1) 
#         self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)

#         self.fc = nn.Linear(256 * 23 * 40, 512) 
        
#         # Output: steer, throttle, brake
#         self.steer_head = nn.Linear(512, 1)
#         self.throttle_head = nn.Linear(512, 1)  
#         self.brake_head = nn.Linear(512, 1)    

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
        
#         x = x.view(x.size(0), -1)  
#         x = F.relu(self.fc(x))
        
#         steer = self.steer_head(x)
#         throttle = self.throttle_head(x)
#         brake = self.brake_head(x)
        
#         return torch.cat([steer, throttle, brake], dim=1)




## RESNET

from torchvision.models import resnet18

class ImitationResNet(nn.Module):
    def __init__(self, pretrained=True):
        super(ImitationResNet, self).__init__()

        base_model = resnet18(pretrained=pretrained)
        
        # Modify the first conv layer to accept 5-channel input (RGB + lane + obs)
        self.conv1 = nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = base_model.bn1
        self.relu = base_model.relu
        self.maxpool = base_model.maxpool
        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4
        self.avgpool = base_model.avgpool

        # Custom heads for control prediction
        self.fc = nn.Linear(512, 512)
        self.steer_head = nn.Linear(512, 1)
        self.throttle_head = nn.Linear(512, 1)
        self.brake_head = nn.Linear(512, 1)

        # Optional: initialize conv1 weights from original 3 channels
        self._init_weights_from_rgb(base_model)

    def _init_weights_from_rgb(self, base_model):
        # Copy RGB weights and zero out extra channels
        old_weights = base_model.conv1.weight.data
        new_weights = torch.zeros((64, 5, 7, 7))  # (out_channels, in_channels, H, W)
        new_weights[:, :3, :, :] = old_weights
        self.conv1.weight.data = new_weights

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc(x))
        
        steer = self.steer_head(x)
        throttle = self.throttle_head(x)
        brake = self.brake_head(x)

        return torch.cat([steer, throttle, brake], dim=1)


In [ ]:
rgb_dir = 'Dataset/rgb_image'
seg_dir = 'Dataset/seg_image'
log_path = 'logs/logs.json'
checkpoint_path = 'checkpoints/last_epoch.pth'

os.makedirs('checkpoints', exist_ok=True)

dataset = ImitationDataset(rgb_dir, seg_dir, log_path, None)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_set, 32, shuffle=True)
val_loader = DataLoader(val_set, 32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = ImitationCNN().to(device)
model = ImitationResNet(pretrained=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

start_epoch = 0
num_epochs = 100
best_rmse = float('inf')

if os.path.exists(checkpoint_path):
    print("Resuming training from last checkpoint")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    train_losses = checkpoint['train_losses']
    val_steer = checkpoint['val_steer']
    val_throttle = checkpoint['val_throttle']
    val_brake = checkpoint['val_brake']
else:
    print("Starting training, no model found")
    train_losses = []
    val_steer, val_throttle, val_brake = [], [], []

for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs= model(inputs)
        
        loss_steer = F.mse_loss(outputs[:, 0], targets[:, 0])
        loss_throttle = F.smooth_l1_loss(outputs[:, 1], targets[:, 1])
        loss_brake = F.smooth_l1_loss(outputs[:, 2], targets[:, 2])
        loss = loss_steer + 2.0 * (loss_throttle + loss_brake) 
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    
    model.eval()
    total_se, total_th, total_br = 0, 0, 0
    n=0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            
            total_se += F.mse_loss(outputs[:, 0], targets[:, 0], reduction='sum').item()
            total_th += F.mse_loss(outputs[:, 1], targets[:, 1], reduction='sum').item()
            total_br += F.mse_loss(outputs[:, 2], targets[:, 2], reduction='sum').item()
            n+= inputs.size(0)
            
    val_rmse = ((total_se +total_th + total_br)/(3*n)) **0.5
    val_st = (total_se/n)**0.5     
    val_th = (total_th/n)**0.5     
    val_br = (total_br/n)**0.5
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses,
            'val_steer': val_steer,
            'val_throttle': val_throttle,
            'val_brake': val_brake
        }, 'checkpoints/last_epoch.pth')

    
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'val_steer': val_steer,
        'val_throttle': val_throttle,
        'val_brake': val_brake
    }, 'models/bc_model.pth')

    
    
    train_losses.append(running_loss/len(train_loader))
    val_steer.append(val_st)
    val_throttle.append(val_th)
    val_brake.append(val_br)
    
    
    print(f"[VAL] Steer MSE: {val_st:.4f}, Throttle RMSE: {val_th:.4f}, Brake RMSE: {val_br:.4f}")
    
    with open('logs/val_metrics.csv', 'a') as f:
        f.write(f"{epoch+1},{val_st:.4f},{val_th:.4f},{val_br:.4f}\n")


Resuming training from last checkpoint
Epoch [6/100], Loss: 0.0085
[VAL] Steer MSE: 0.0259, Throttle RMSE: 0.0674, Brake RMSE: 0.0476
Epoch [7/100], Loss: 0.0056
[VAL] Steer MSE: 0.0263, Throttle RMSE: 0.0634, Brake RMSE: 0.0504
Epoch [8/100], Loss: 0.0035
[VAL] Steer MSE: 0.0250, Throttle RMSE: 0.0668, Brake RMSE: 0.0549
Epoch [9/100], Loss: 0.0025
[VAL] Steer MSE: 0.0261, Throttle RMSE: 0.0655, Brake RMSE: 0.0482
Epoch [10/100], Loss: 0.0017
[VAL] Steer MSE: 0.0247, Throttle RMSE: 0.0635, Brake RMSE: 0.0468
Epoch [11/100], Loss: 0.0012
[VAL] Steer MSE: 0.0250, Throttle RMSE: 0.0648, Brake RMSE: 0.0468
Epoch [12/100], Loss: 0.0009
[VAL] Steer MSE: 0.0254, Throttle RMSE: 0.0652, Brake RMSE: 0.0497
Epoch [13/100], Loss: 0.0009
[VAL] Steer MSE: 0.0253, Throttle RMSE: 0.0653, Brake RMSE: 0.0492
Epoch [14/100], Loss: 0.0009
[VAL] Steer MSE: 0.0253, Throttle RMSE: 0.0654, Brake RMSE: 0.0477
Epoch [15/100], Loss: 0.0007
[VAL] Steer MSE: 0.0254, Throttle RMSE: 0.0693, Brake RMSE: 0.0504
Epoch